<a href="https://colab.research.google.com/github/Rahul-Reddy-M/Diabetes-Risk-Checker-Tool/blob/main/Diabetes_Risk_Checker_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!ngrok config add-authtoken Authtoken

/bin/bash: line 1: ngrok: command not found


In [7]:
# Install flask-ngrok
!pip install flask-ngrok

In [8]:
!pip install Flask --upgrade

  Using cached flask-3.0.0-py3-none-any.whl (99 kB)
  Using cached blinker-1.7.0-py3-none-any.whl (13 kB)
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [9]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok

# Load the dataset
data = pd.read_csv("diabetes.csv")

# Split the data into features (X) and labels (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Normalize the features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

# Define the optimized neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(8,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=150, batch_size=32)

# Initialize Flask app
app = Flask(__name__, template_folder='/content/drive/MyDrive/templates')
run_with_ngrok(app)  # Start ngrok when the app is run

# Define a function to predict risk
def predict_risk(user_input):
    # Normalize user input using the same scaler as before
    user_input = sc.transform(user_input)
    risk_score = model.predict(user_input)
    return risk_score

# Define a function to provide general health tips based on risk score
def get_health_tips(risk_score):
    if risk_score > 0.5:
        return "You are at a high risk of diabetes complications. Consider consulting a healthcare professional."
    else:
        return "Your risk of diabetes complications is low. However, maintaining a healthy lifestyle is important."

# Create an HTML template for the input form
@app.route("/", methods=["GET", "POST"])
def diabetes_checker():
    if request.method == "POST":
        user_input = [float(request.form["age"]),
                      float(request.form["pregnancies"]),
                      float(request.form["glucose"]),
                      float(request.form["blood_pressure"]),
                      float(request.form["skin_thickness"]),
                      float(request.form["insulin"]),
                      float(request.form["bmi"]),
                      float(request.form["diabetes_pedigree"])]

        risk_score = predict_risk([user_input])
        health_tips = get_health_tips(risk_score)

        return render_template("result.html", risk_score=risk_score[0][0], health_tips=health_tips)

    return render_template("index.html")

if __name__ == "__main__":
    app.run()


Epoch 1/150
24/24 [==============================] - 2s 6ms/step - loss: 0.6691 - accuracy: 0.6315
Epoch 2/150
24/24 [==============================] - 0s 3ms/step - loss: 0.5846 - accuracy: 0.7018
Epoch 3/150
24/24 [==============================] - 0s 3ms/step - loss: 0.5471 - accuracy: 0.7500
Epoch 4/150
24/24 [==============================] - 0s 3ms/step - loss: 0.5254 - accuracy: 0.7357
Epoch 5/150
24/24 [==============================] - 0s 3ms/step - loss: 0.5011 - accuracy: 0.7656
Epoch 6/150
24/24 [==============================] - 0s 3ms/step - loss: 0.5019 - accuracy: 0.7669
Epoch 7/150
24/24 [==============================] - 0s 3ms/step - loss: 0.4836 - accuracy: 0.7669
Epoch 8/150
24/24 [==============================] - 0s 2ms/step - loss: 0.4663 - accuracy: 0.7799
Epoch 9/150
24/24 [==============================] - 0s 3ms/step - loss: 0.4678 - accuracy: 0.7786
Epoch 10/150
24/24 [==============================] - 0s 3ms/step - loss: 0.4611 - accuracy: 0.7891
Epoch 11/

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://0fb4-34-121-101-187.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
